# 3.5 Analysing Data

Exploring data using the MongoDB aggregation framework: initial analysis and additional cleaning. Used to create data processing pipelines.

e.g. Twitter Data Set

Examples of questions: Understand behaviour of users and networks involved.

Aggregation Framework provides a powerful tool for analysing data.
E.g.: Determine which user has produced the most tweets. 
Process:
1. Group tweets by user
2. Count each user's tweets
3. Sort into descending order (of number of tweets)
4. Select user at the top (the one with most tweets)

Aggregation queries in MongoDB issued using 'aggregate', done using a pipeline.

In [ ]:
from pymongo import MongoClient
import pprint

client = MongoClient("mongodb://localhost:27017")
db = client.twitter

def most_tweets():
    # Issue aggregation query
    result = db.tweets.aggregate([
            # For user subdocument, I want the screen name field
            # "$user.screen_name" don't make it a string. Not an operator.
            # Want value of user.screen_name
            { "$group" : { "_id" : "$user.screen_name",
                          # Accumulator operator "$sum": For all docs that have the same value for _id,
                          # Increment count by 1.
                           "count" : { "$sum" : 1 } } },
            # Sort docs passed into this stage (output of "$group")
            # based on the count in descending order
            { "$sort" : { "count" : -1} } ])
    return result

if __name__ = "__main__":
    result = most_tweets()
    pprint.pprint(result)

### Aggregation Pipline 

* diagram
* e.g. "\$group" -> "\$sort" 
* Collection fed into group stage. Finds tweets per user and accumulates them.
* Depending on which operator is used in a given stage, stage may be reshaping data. Collection of tweets have dozens of fields, putting through "\$group" stage turns it into data with 2 fields.
* Use aggregation operators to produce stages.

### Exercise: Fix

In [ ]:
#!/usr/bin/env python
"""
The tweets in our twitter collection have a field called "source". This field describes the application
that was used to create the tweet. Following the examples for using the $group operator, your task is 
to modify the 'make-pipeline' function to identify most used applications for creating tweets. 
As a check on your query, 'web' is listed as the most frequently used application.
'Ubertwitter' is the second most used. The number of counts should be stored in a field named 'count'
(see the assertion at the end of the script).

Please modify only the 'make_pipeline' function so that it creates and returns an aggregation pipeline
that can be passed to the MongoDB aggregate function. As in our examples in this lesson, the aggregation 
pipeline should be a list of one or more dictionary objects. 
Please review the lesson examples if you are unsure of the syntax.

Your code will be run against a MongoDB instance that we have provided. 
If you want to run this code locally on your machine, you have to install MongoDB, 
download and insert the dataset.
For instructions related to MongoDB setup and datasets please see Course Materials.

Please note that the dataset you are using here is a smaller version of the twitter dataset 
used in examples in this lesson. 
If you attempt some of the same queries that we looked at in the lesson examples,
your results will be different.
"""


def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{"$group" : {"source" : "$source",
                             "count" : {"$sum" : 1} } },
                {"$sort" : {"count" : -1} } ]
    return pipeline

def tweet_sources(db, pipeline):
    return [doc for doc in db.tweets.aggregate(pipeline)]

if __name__ == '__main__':
    db = get_db('twitter')
    pipeline = make_pipeline()
    result = tweet_sources(db, pipeline)
    import pprint
    pprint.pprint(result[0])
    assert result[0] == {u'count': 868, u'_id': u'web'}


## Aggregation Operators: An overview

Previously introduced **group** and **sort**.

### 1. \$project
Reshaping data for the next stage.
* e.g. Selecting which fields you are interested in, regardless of where they are nested.

Functions:
* Include fields from  the original document
* Insert computed fileds (e.g. produce ratio based on two numeeric fields)
* Rename fields
* Create fields that hold subdocuments 

### 2. \$match
Filters documents

### 3. \$skip
Skip over a certain number of documents at the beginning of the input set of documents.

### 4. \$limit

Limit is opposite of skipe. Limit to first 3 documents of input to next stage, say.

### 5. \$unwind

(Recall fields can have arrays as values.)
* For every value of an array field, it will create an instance of the document containing that array field for every value in the array.
* Can run groupby in next stage of the pipeline
 where we care about individual values and group based on those values.
 * e.g. group based on hashtags included in tweets. 
 
For more operators see [Project operator documentation](http://docs.mongodb.org/manual/reference/operator/aggregation/project/#pipe._S_project)

#### Question to work on
Who has the highest followers to friends (followers to following) ratio? (Twitter dataset)

In [ ]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client.examples

def highest_ratio():
    result = db.tweets.aggregate([
            # Specify constraints on type of doc we want to let through match stage.
            # I.e. those who have both a friends and followers count >0
            # Specifying field, inequality operator, value 0.
            {"$match": {"user.friends_count": {"$gt": 0},
                        "user.followers_count": {"$gt": 0}}},
            # Ratio field has value followers_count/friends_count
            {"$project": {"ratio": {"$divide": ["$user.followers_count",
                                                "$user.friends_count"]},
                                    # Dollar sign because not string literal
                                    "screen_name": "$user.screen_name"}},
            # Here, docs will have precisely two fields: screen_name and ratio.
            # Sort in descending ordor of ratio (descending -> -1)
            {"$sort": {"ratio": -1}},
            # We only want the top (largest) entry
            {"$limit": 1}
        ])
    return result

# result = highest_ratio()

# Returns array-valued field with three fields '_id', 'ratio', 'screen_name'


In [ ]:
# Quiz: Using match and project

#!/usr/bin/env python
"""
Write an aggregation query to answer this question:

Of the users in the "Brasilia" timezone who have tweeted 100 times or more,
who has the largest number of followers?

The following hints will help you solve this problem:
- Time zone is found in the "time_zone" field of the user object in each tweet.
- The number of tweets for each user is found in the "statuses_count" field.
  To access these fields you will need to use dot notation (from Lesson 4)
- Your aggregation query should return something like the following:
{u'ok': 1.0,
 u'result': [{u'_id': ObjectId('52fd2490bac3fa1975477702'),
                  u'followers': 2597,
                  u'screen_name': u'marbles',
                  u'tweets': 12334}]}
Note that you will need to create the fields 'followers', 'screen_name' and 'tweets'.

Please modify only the 'make_pipeline' function so that it creates and returns an aggregation 
pipeline that can be passed to the MongoDB aggregate function. As in our examples in this lesson,
the aggregation pipeline should be a list of one or more dictionary objects. 
Please review the lesson examples if you are unsure of the syntax.

Your code will be run against a MongoDB instance that we have provided. If you want to run this code
locally on your machine, you have to install MongoDB, download and insert the dataset.
For instructions related to MongoDB setup and datasets please see Course Materials.

Please note that the dataset you are using here is a smaller version of the twitter dataset used 
in examples in this lesson. If you attempt some of the same queries that we looked at in the lesson 
examples, your results will be different.
"""

def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    # complete the aggregation pipeline
    """
    Of the users in the "Brasilia" timezone who have tweeted 100 times or more,
    who has the largest number of followers?
    """
    pipeline = [
        # Users in "Brasilia" timezone who have tweeted 100 times or more
        {"$match": {"user.time_zone": "Brasilia",
                    "user.statuses_count": {"$gte": 100}}},
        # Number of followers
        {"$project": {"followers": "$user.followers_count",
                      "screen_name": "$user.screen_name",
                      "tweets": "$user.statuses_count"}},
        # Sort by number of followers (Descending)
        {"$sort": {"followers": -1}},
        # Only highest (first)
        {"$limit": 1}
    ]
    return pipeline

def highest_ratio():
    result = db.tweets.aggregate([
            # Specify constraints on type of doc we want to let through match stage.
            # I.e. those who have both a friends and followers count >0
            # Specifying field, inequality operator, value 0.
            {"$match": {"user.friends_count": {"$gt": 0},
                        "user.followers_count": {"$gt": 0}}},
            # Ratio field has value followers_count/friends_count
            {"$project": {"ratio": {"$divide": ["$user.followers_count",
                                                "$user.friends_count"]},
                                    # Dollar sign because not string literal
                                    "screen_name": "$user.screen_name"}},
            # Here, docs will have precisely two fields: screen_name and ratio.
            # Sort in descending ordor of ratio (descending -> -1)
            {"$sort": {"ratio": -1}},
            # We only want the top (largest) entry
            {"$limit": 1}
        ])
    return result

def aggregate(db, pipeline):
    return [doc for doc in db.tweets.aggregate(pipeline)]


if __name__ == '__main__':
    db = get_db('twitter')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    pprint.pprint(result)
    assert len(result) == 1
    assert result[0]["followers"] == 17209

#### More on \$unwind operator

 E.g. who included the most user mentions? (included inside the entities subdocument 


In [ ]:
from pymongo import MongoClient
import pprint

client = MongoClient("mongodb://localhost:27017")
db = client.examples

def user_mentions():
    result = db.tweets.aggregate([
            {"$unwind": "$entities.user_mentions"},
            # Group docs that passed through first stage by user who created the tweet
            # Produce a count field that increments counter each time it sees the document that has the same screen name
            # Counts all mentions, not unique mentions. I.e. if I mention @a twice in my tweet, it's counted twice.
            {"$group": {"_id": "$user.screen_name",
                        "count": {"$sum": 1}}},
            # Sort in descending ordor of count (descending -> -1)
            {"$sort": {"count": -1}},
            # We only want the top (largest) entry
            {"$limit": 1}
        ])
    return result

# High speed of execution of queries because this functionality is fundamental to server itself.

# result = user_mentions()
# pprint.pprint(result)

In [ ]:
# Quiz: Using unwind
#!/usr/bin/env python
"""
For this exercise, let's return to our cities infobox dataset. The question we would like you to answer
is as follows:  Which region or district in India contains the most cities? (Make sure that the count of
cities is stored in a field named 'count'; see the assertions at the end of the script.)

As a starting point, use the solution for the example question we looked at -- "Who includes the most
user mentions in their tweets?"

One thing to note about the cities data is that the "isPartOf" field contains an array of regions or 
districts in which a given city is found. See the example document in Instructor Comments below.

Please modify only the 'make_pipeline' function so that it creates and returns an aggregation pipeline 
that can be passed to the MongoDB aggregate function. As in our examples in this lesson, the aggregation 
pipeline should be a list of one or more dictionary objects. Please review the lesson examples if you 
are unsure of the syntax.

Your code will be run against a MongoDB instance that we have provided. If you want to run this code 
locally on your machine, you have to install MongoDB, download and insert the dataset.
For instructions related to MongoDB setup and datasets please see Course Materials.

Please note that the dataset you are using here is a smaller version of the cities collection used in 
examples in this lesson. If you attempt some of the same queries that we looked at in the lesson 
examples, your results may be different.
"""

def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db



def make_pipeline():
    """
    Which region or district in India contains the most cities? (Make sure that the count of
    cities is stored in a field named 'count'; see the assertions at the end of the script.)
    """
    # complete the aggregation pipeline
    pipeline = [
        # We want only entries with country = India
        {"$match": {"country": "India"}},
        # Unwind to separate isPartOf (regions)
        {"$unwind": "$isPartOf"},
        # Group by region (isPartOf) and count
        {"$group": {"_id": "$isPartOf",
                    "count": {"$sum": 1}}},
        # Sort by count (descending)
        {"$sort": {"$count": -1}},
        # Return region with highest count
        {"$limit": 1}
    ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.cities.aggregate(pipeline)]

if __name__ == '__main__':
    db = get_db('examples')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    print "Printing the first result:"
    import pprint
    pprint.pprint(result[0])
    assert result[0]["_id"] == "Uttar Pradesh"
    assert result[0]["count"] == 623




### Help

### \$group (more detail)

Aggregates input in some way based on operator specified.

* \$sum
* \$first (Selects first documented group)
* \$last
* \$max
* \$min
* \$avg

Question to investigate: Calculate average number of retweets for any tweet based on the hashtag.

In [ ]:
from pymongo import MongoClient
import pprint

client = MongoClient("mongodb://localhost:27017")
db = client.examples

def hashtag_retweet_avg():
    result = db.tweets.aggregate([
            {"$unwind": "$entities.hashtags"},
            # Aggregate based on hashtag itself
            {"$group": {"_id": "$entities.hashtags.text",
                        "retweet_avg": {"$avg": "$retweet_count"}
                       }},
            # Sort in descending ordor of count (descending -> -1)
            {"$sort": {"retweet_avg": -1}}
        ])
    return result

# result = user_mentions()
# pprint.pprint(result)

More group operators. These deal with arrays.
*\$push
*\$addToSet: Adds values to an array by treating it as a set, i.e. won't add same value more than once


In [ ]:
from pymongo import MongoClient
import pprint

client = MongoClient("mongodb://localhost:27017")
db = client.examples

def unique_hashtags_by_user():
    result = db.tweets.aggregate([
            {"$unwind": "$entities.hashtags"},
            # Aggregate based on hashtag itself
            {"$group": {"_id": "$user.screen_name",
                        "unique_hashtags": {
                        # Each unique hashtag added precisely once
                        "$addToSet": "$entities.hashtags.text"}
                       }},
            # Sort in descending ordor of count (descending -> -1)
            {"$sort": {"_id": -1}}
        ])
    return result

### \$push

In [ ]:
# Using push

#!/usr/bin/env python
"""
$push is similar to $addToSet. The difference is that rather than accumulating only unique values 
it aggregates all values into an array.

Using an aggregation query, count the number of tweets for each user. In the same $group stage, 
use $push to accumulate all the tweet texts for each user. Limit your output to the 5 users
with the most tweets. 
Your result documents should include only the fields:
"_id" (screen name of user), 
"count" (number of tweets found for the user),
"tweet_texts" (a list of the tweet texts found for the user).  

Please modify only the 'make_pipeline' function so that it creates and returns an aggregation 
pipeline that can be passed to the MongoDB aggregate function. As in our examples in this lesson, 
the aggregation pipeline should be a list of one or more dictionary objects. 
Please review the lesson examples if you are unsure of the syntax.

Your code will be run against a MongoDB instance that we have provided. If you want to run this code 
locally on your machine, you have to install MongoDB, download and insert the dataset.
For instructions related to MongoDB setup and datasets please see Course Materials.

Please note that the dataset you are using here is a smaller version of the twitter dataset used in 
examples in this lesson. If you attempt some of the same queries that we looked at in the lesson 
examples, your results will be different.
"""

def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [
        # Count number of tweets for each user: group tweets by user
        {"$group": {"_id": "$user.screen_name", 
                    "count": {"$sum": 1},
                    # Accumulate tweet texts
                    "tweet_texts": {"$push": "$text.text"}}},
        # Limit to 5 users with most tweets
        {"$sort": {"count": -1}},
        {"$limit": 5}
    ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.twitter.aggregate(pipeline)]


if __name__ == '__main__':
    db = get_db('twitter')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    pprint.pprint(result)
    assert len(result) == 5
    assert result[0]["count"] > result[4]["count"]
    

### Multiple stages using a given operator

Question: Who has mentioned the most (unique users)?



In [ ]:
from pymongo import MongoClient
import pprint

client = MongoClient("mongodb://localhost:27017")
db = client.examples

def unique_user_mentions():
    result = db.tweets.aggregate([
            {"$unwind": "$entities.user_mentions"},
            # Aggregating on user's screen name. Use $addToSet of user mentioned.
            # i.e. unique set of users mentioned in tweets produced by each user.
            {"$group": {
                    "_id": "$user.screen_name",
                    # mset stands for 'mentions set'
                    "mset": {
                        "$addToSet": "$entities.user_mentions.screen_name"
                    }}},
            # Now we have a list of unique users. Haven't counted them yet.
            # Unwind generates one doc for every unique user mentioned per user
            {"$unwind": "$mset"},
            # Calculate count that we want. $_id directs to same _id as before, i.e. $user.screen_name
            {"$group": {"_id": "$_id", "count": {"$sum": 1}}}
            {"$sort": {"count": -1}},
            {"$limit": 10}
        ])
    return result

# High speed of execution of queries because this functionality is fundamental to server itself.

# result = user_mentions()
# pprint.pprint(result)

In [ ]:
# Quiz: Same Operator

#!/usr/bin/env python
"""
In an earlier exercise we looked at the cities dataset and asked which region in India contains 
the most cities. In this exercise, we'd like you to answer a related question regarding regions in 
India. What is the average city population for a region in India? Calculate your answer by first 
finding the average population of cities in each region and then by calculating the average of the 
regional averages.

Hint: If you want to accumulate using values from all input documents to a group stage, you may use 
a constant as the value of the "_id" field. For example, 
    { "$group" : {"_id" : "India Regional City Population Average",
      ... }

Please modify only the 'make_pipeline' function so that it creates and returns an aggregation 
pipeline that can be passed to the MongoDB aggregate function. As in our examples in this lesson, 
the aggregation pipeline should be a list of one or more dictionary objects. 
Please review the lesson examples if you are unsure of the syntax.

Your code will be run against a MongoDB instance that we have provided. If you want to run this code 
locally on your machine, you have to install MongoDB, download and insert the dataset.
For instructions related to MongoDB setup and datasets please see Course Materials.

Please note that the dataset you are using here is a smaller version of the twitter dataset used 
in examples in this lesson. If you attempt some of the same queries that we looked at in the lesson 
examples, your results will be different.
"""

def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    """
    What is the average city population for a region in India? Calculate your answer by first 
    finding the average population of cities in each region and then by calculating the average of the 
    regional averages.
    """
    # complete the aggregation pipeline
    pipeline = [
        # Unwind by "isPartOf"
        {"$unwind": "$isPartOf"},
        # Group by region ("isPartOf") and calculate average pop of cities in each region
        {"$group": {"_id": "$isPartOf",
                    "average_city_pop": {"$avg": "$population"}}},
        # Calculate average of all regional averages
        {"$group": {"_id": "Average city pop for a region in India",
                    "avg": {"$avg": "$average_city_pop"}}}
    ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.cities.aggregate(pipeline)]


if __name__ == '__main__':
    db = get_db('examples')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    assert len(result) == 1
    # Your result should be close to the value after the minus sign.
    assert abs(result[0]["avg"] - 196025.97814809752) < 10 ** -8
    import pprint
    pprint.pprint(result)



### Don't know why code above is wrong.

## Database Performance: Driven by 'Use Indexes or Not'

Basics of indexing

DB stores data in large files on disk. 
Pull out document using a query. By default DB scans through the entire collection to find the data. (Table scan in rel DB or collection scan in MongDB.) -> Death for performance.

Instead, create an index or more than one index.

**How it works**

If something is ordered, can use binary search to do it.
Specify a key.
Index is an ordered key.
Not stored linearly in MongoDB, stored using a B tree. But conceptually like the picture.

Indexes are ordered lists of keys. You can have just one or you can construct one out of e.g. 3 keys (Hashtag, date, username). Order matters because conceptually the index is built by level. Here, within hashtags you have dates, within dates you have users.

(Diagram 2)

In order for MongoDB to use an index, you have to give it a leftmost set of items. In above example, just giving a date won't be useful.

Every time you insert something into the database, the index needs to be updated so it takes time. Need to take this into consideration when designing indices.

### Using indexes

More on indexing: MongoDB University or MongoDB documentation on Indexes.

### Geospatial Indexes
Queries for location near a location, e.g. looking for a nearby cafe.

Here look at 2D. MongoDB also supports spherical geospatial indexes.

Find other items that are close to query location (x,y).

Three steps:
1. Location: [x,y] (stored as an array.)
2. Call "ensureIndex" to create an index on this field. i.e. ensureIndex({'location':...})
3. Query using \$near operator.

In PyMongo, pass a list of tuples for ensure_index
self.client.osm.nodes.ensure_index([('loc', pymongo.GEO2D)]}

Treat pymongo.GEO2D as a direction argument. Not a string.

In [ ]:
# 2:27 in vid, OSM code.

## Problem Set

### 1. Most Common City Name

In [ ]:
#!/usr/bin/env python
"""
Use an aggregation query to answer the following question. 

What is the most common city name in our cities collection?

Your first attempt probably identified None as the most frequently occurring
city name. What that actually means is that there are a number of cities
without a name field at all. It's strange that such documents would exist in
this collection and, depending on your situation, might actually warrant
further cleaning. 

To solve this problem the right way, we should really ignore cities that don't
have a name specified. As a hint ask yourself what pipeline operator allows us
to simply filter input? How do we test for the existence of a field?

Please modify only the 'make_pipeline' function so that it creates and returns
an aggregation pipeline that can be passed to the MongoDB aggregate function.
As in our examples in this lesson, the aggregation pipeline should be a list of
one or more dictionary objects. Please review the lesson examples if you are
unsure of the syntax.

Your code will be run against a MongoDB instance that we have provided. If you
want to run this code locally on your machine, you have to install MongoDB, 
download and insert the dataset. For instructions related to MongoDB setup and
datasets please see Course Materials.

Please note that the dataset you are using here is a different version of the
cities collection provided in the course materials. If you attempt some of the
same queries that we look at in the problem set, your results may be different.
"""

def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    # complete the aggregation pipeline
    """
    What is the most common city name in our cities collection?
    """
    pipeline = [ 
        # Search only cities that have names
        {"$match": {"$name": {"$exists": 1}}},
        # Group cities with the same name and count number in each group
        {"$group": {"_id": "$name",
                    "count": {"$sum": 1}}},
        # Sort groups by descending order
        {"$sort": {"count": -1}},
        # Top entry
        {"$limit": 1}
    ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.cities.aggregate(pipeline)]


if __name__ == '__main__':
    # The following statements will be used to test your code by the grader.
    # Any modifications to the code past this point will not be reflected by
    # the Test Run.
    db = get_db('examples')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    pprint.pprint(result[0])
    assert len(result) == 1
    assert result[0] == {'_id': 'Shahpur', 'count': 6}


### Mistakes:

* "\$fieldname" instead of "\$fieldname.text"
* Extra curly or square brackets
* Forgot quotes